In [1]:
import glob
import os
import shutil

from astropy.io import fits
from astropy import stats
from astropy.time import Time
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

# Determine where the visualization will be rendered
output_notebook()

Loading BokehJS ...

In [20]:
subarray_mean = pd.read_pickle('Feb_2022_all_sub_pf_and_mean_data.pkl')
subarray_mean

,filename,propid,path,obsdate,obstime,shutter,flash_lvl,flash_dur,flash_cur,subarray,exp_time,mean,sig_mean
rootname,,,,,,,,,,,,,
ic6de2u9q,ic6de2u9q_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-03-09,13:59:17,B,12,4.7,LOW,True,0.5,7.312437,7.048593
icha24imq,icha24imq_flc.fits,13560,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2014-06-29,04:16:36,A,3,1.2,LOW,True,0.5,2.170188,1.976714
idp530aiq,idp530aiq_flc.fits,14984,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2018-06-22,04:55:12,B,15,0.2,MED,True,0.5,8.972477,8.680973
icrz19f7q,icrz19f7q_flc.fits,14006,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2015-05-25,22:31:24,B,12,4.7,LOW,True,0.5,7.337009,7.018181
idp531a7q,idp531a7q_flc.fits,14984,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2018-07-20,04:24:43,B,12,4.7,LOW,True,0.5,7.372123,7.059274
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ieg839idq,ieg839idq_flc.fits,16398,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2021-09-07,05:39:43,B,15,6.3,LOW,True,0.5,9.602947,9.375117
idp524egq,idp524egq_flc.fits,14984,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2018-04-28,02:12:18,B,15,0.2,MED,True,0.5,9.064358,8.738423
icrz24ubq,icrz24ubq_flc.fits,14006,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2015-06-23,19:35:12,B,5,2.0,LOW,True,0.5,3.178271,2.902492


In [21]:
low_sub_pf_isr_13078_nov15 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['propid'] == 13078) & (subarray_mean['shutter'] == 'A') & (subarray_mean['obsdate'] == '2012-11-15')]
low_sub_pf_isr_13078_nov16 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['propid'] == 13078) & (subarray_mean['shutter'] == 'A') & (subarray_mean['obsdate'] == '2012-11-16')]

both_dates = [low_sub_pf_isr_13078_nov15, low_sub_pf_isr_13078_nov16]

low_sub_pf_isr_13078 = pd.concat(both_dates)

From 2012 ISR: "The data in Figure 10 are from program 13078 visits A3, A6, A7, and A8 and taken on November 15
and 16, 2012. Images were combined in pairs with STSDAS CRREJ, and the mean pixel value was
computed for the 1024x1024 pixel area nearest the C amplifier using IMSTAT with 3 iterations of 3
sigma rejection. "

Run the cell below to calculate the sigma clipped means:

In [22]:

paths = low_sub_pf_isr_13078.path.tolist() 

means = []
sig_means = []
for f in paths:
    path = f
    data = fits.getdata(path)
    data_clip_1 = stats.sigma_clip(data, 3)
    data_clip_2 = stats.sigma_clip(data_clip_1, 3)
    data_clip_3 = stats.sigma_clip(data_clip_2, 3)
    mean = np.mean(data)
    sig_mean = np.mean(data_clip_3)
    means.append(mean)
    sig_means.append(sig_mean)

dict = {'path': paths, 'mean': means, 'sig_mean': sig_means}
path_and_mean_sub = pd.DataFrame(dict)

FileNotFoundError: [Errno 2] No such file or directory: '/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/ic6da3upq/ic6da3upq_flc.fits'

In [23]:
# This count rate brings the values to a similar but not the same value as ISR. And the use of the flash duration in the calculation seems out of place?
#low_sub_pf_isr_13078['count_rate'] = (((low_sub_pf_isr_13078['sig_mean'])/1.55)/low_sub_pf_isr_13078['exp_time'])/low_sub_pf_isr_13078['flash_dur']

low_sub_pf_isr_13078['count_rate'] = (((low_sub_pf_isr_13078['sig_mean'])*1.55)/low_sub_pf_isr_13078['flash_dur'])

In [24]:
display(low_sub_pf_isr_13078)

,filename,propid,path,obsdate,obstime,shutter,flash_lvl,flash_dur,flash_cur,subarray,exp_time,mean,sig_mean,count_rate
rootname,,,,,,,,,,,,,,
ic6da3upq,ic6da3upq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,04:54:05,A,10,3.9,LOW,True,0.5,6.872035,6.259343,2.487688
ic6da6wqq,ic6da6wqq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,18:11:21,A,3,1.2,LOW,True,0.5,2.355810,2.006855,2.592188
ic6da3umq,ic6da3umq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,04:49:30,A,3,1.2,LOW,True,0.5,2.509913,1.991227,2.572002
ic6da3unq,ic6da3unq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,04:51:01,A,5,2.0,LOW,True,0.5,3.622655,3.233530,2.505986
ic6da6wtq,ic6da6wtq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,18:15:56,A,10,3.9,LOW,True,0.5,6.608037,6.257084,2.486790
ic6daavsq,ic6daavsq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,15:22:20,A,12,4.7,LOW,True,0.5,7.678378,7.451395,2.457375
ic6da6wsq,ic6da6wsq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,18:14:23,A,8,3.1,LOW,True,0.5,5.592704,5.007172,2.503586
ic6da9wjq,ic6da9wjq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,17:22:22,A,12,4.7,LOW,True,0.5,7.857969,7.504175,2.474781
ic6da3urq,ic6da3urq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,04:57:08,A,20,7.7,LOW,True,0.5,12.536662,12.137565,2.443276


In [25]:
low_sub_pf_isr_13078.to_csv('countrate_for_2012_isr_data.csv')

In [26]:
low_sub_pf_15718 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['shutter'] == 'A') & (subarray_mean['propid'] == 15718) ]
low_sub_pf_15718
low_sub_pf_16398 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['shutter'] == 'A') & (subarray_mean['propid'] == 16398) ]
low_sub_pf_15718['obsdate'].unique()


array(['2020-06-19', '2020-05-19', '2020-03-22', '2020-08-11',
       '2019-11-10', '2020-04-21', '2020-09-08', '2020-01-02',
       '2020-07-18', '2019-12-03', '2020-08-03', '2020-10-06',
       '2019-12-05', '2020-02-25', '2020-01-28', '2020-01-01',
       '2020-02-26', '2020-06-18', '2019-12-04'], dtype=object)

In [27]:
low_sub_pf_2020_mar = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['shutter'] == 'A') & (subarray_mean['propid'] == 15718) & (subarray_mean['obsdate'] == '2020-03-22')]
low_sub_pf_2020_apr = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['shutter'] == 'A') & (subarray_mean['propid'] == 15718) & (subarray_mean['obsdate'] == '2020-04-21')]
low_sub_pf_2020_feb = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['shutter'] == 'A') & (subarray_mean['propid'] == 15718) & (subarray_mean['obsdate'] == '2020-02-25')]
low_sub_pf_2020_jan = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['shutter'] == 'A') & (subarray_mean['propid'] == 15718) & (subarray_mean['obsdate'] == '2020-01-28')]
low_sub_pf_2020_may = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['shutter'] == 'A') & (subarray_mean['propid'] == 15718) & (subarray_mean['obsdate'] == '2020-05-19')]
low_sub_pf_2020_jun = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['shutter'] == 'A') & (subarray_mean['propid'] == 15718) & (subarray_mean['obsdate'] == '2020-06-19')]
low_sub_pf_2020_jul = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['shutter'] == 'A') & (subarray_mean['propid'] == 15718) & (subarray_mean['obsdate'] == '2020-07-18')]
low_sub_pf_2020_aug = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['shutter'] == 'A') & (subarray_mean['propid'] == 15718) & (subarray_mean['obsdate'] == '2020-08-03')]
low_sub_pf_2020_sep = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['shutter'] == 'A') & (subarray_mean['propid'] == 15718) & (subarray_mean['obsdate'] == '2020-09-08')]
low_sub_pf_2020_oct = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['shutter'] == 'A') & (subarray_mean['propid'] == 15718) & (subarray_mean['obsdate'] == '2020-10-06')]


frames = [low_sub_pf_2020_mar, low_sub_pf_2020_apr,low_sub_pf_2020_feb,low_sub_pf_2020_jan,low_sub_pf_2020_may,low_sub_pf_2020_jun,low_sub_pf_2020_jul,low_sub_pf_2020_aug,
          low_sub_pf_2020_sep,low_sub_pf_2020_oct]
low_sub_pf_2020 = pd.concat(frames)

In [28]:
#low_sub_pf_2020['count_rate'] = (((low_sub_pf_2020['sig_mean'])/1.55)/low_sub_pf_2020['exp_time'])/low_sub_pf_2020['flash_dur']
low_sub_pf_2020['count_rate'] = (((low_sub_pf_2020['sig_mean'])*1.55)/low_sub_pf_2020['flash_dur'])

In [29]:
display(low_sub_pf_2020)

,filename,propid,path,obsdate,obstime,shutter,flash_lvl,flash_dur,flash_cur,subarray,exp_time,mean,sig_mean,count_rate
rootname,,,,,,,,,,,,,,
ie1b21elq,ie1b21elq_flc.fits,15718,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2020-03-22,04:57:34,A,5,2.0,LOW,True,0.5,3.487203,3.148278,2.439916
ie1b20foq,ie1b20foq_flc.fits,15718,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2020-03-22,09:57:22,A,12,4.7,LOW,True,0.5,7.669121,7.328719,2.416918
ie1b21emq,ie1b21emq_flc.fits,15718,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2020-03-22,04:59:05,A,8,3.1,LOW,True,0.5,5.130196,4.872688,2.436344
ie1b21enq,ie1b21enq_flc.fits,15718,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2020-03-22,05:00:38,A,10,3.9,LOW,True,0.5,6.473282,6.102218,2.425240
ie1b19ebq,ie1b19ebq_flc.fits,15718,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2020-03-22,03:19:10,A,12,4.7,LOW,True,0.5,7.734169,7.386820,2.436079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ie1b42l1q,ie1b42l1q_flc.fits,15718,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2020-10-06,05:48:04,A,22,9.2,LOW,True,0.5,15.152306,14.259753,2.402458
ie1b42kyq,ie1b42kyq_flc.fits,15718,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2020-10-06,05:43:16,A,10,4.2,LOW,True,0.5,7.169715,6.618970,2.442715
ie1b42kxq,ie1b42kxq_flc.fits,15718,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2020-10-06,05:41:43,A,8,3.3,LOW,True,0.5,5.764659,5.182280,2.434101


In [30]:
low_sub_pf_2020.to_csv('countrate_for_2020_data.csv')

In [31]:
array(['2020-06-19', '2020-05-19', '2020-03-22', '2020-08-11',
       '2019-11-10', '2020-04-21', '2020-09-08', '2020-01-02',
       '2020-07-18', '2019-12-03', '2020-08-03', '2020-10-06',
       '2019-12-05', '2020-02-25', '2020-01-28', '2020-01-01',
       '2020-02-26', '2020-06-18', '2019-12-04'], dtype=object)

NameError: name 'array' is not defined

In [32]:
low_sub_pf_2020_apr = low_sub_pf_2020.loc[(low_sub_pf_2020['obsdate'] == '2020-04-21')]
low_sub_pf_2020_mar = low_sub_pf_2020.loc[(low_sub_pf_2020['obsdate'] == '2020-03-22')]
low_sub_pf_2020_feb = low_sub_pf_2020.loc[(low_sub_pf_2020['obsdate'] == '2020-02-25')]
low_sub_pf_2020_jan = low_sub_pf_2020.loc[(low_sub_pf_2020['obsdate'] == '2020-01-28')]
low_sub_pf_2020_may = low_sub_pf_2020.loc[(low_sub_pf_2020['obsdate'] == '2020-05-19')]
low_sub_pf_2020_jun = low_sub_pf_2020.loc[(low_sub_pf_2020['obsdate'] == '2020-06-18')]
low_sub_pf_2020_jul = low_sub_pf_2020.loc[(low_sub_pf_2020['obsdate'] == '2020-07-18')]


x = low_sub_pf_2020_apr['flash_dur']
y = low_sub_pf_2020_apr['count_rate']

x_mar = low_sub_pf_2020_mar['flash_dur']
y_mar = low_sub_pf_2020_mar['count_rate'] 

x_feb = low_sub_pf_2020_feb['flash_dur']
y_feb = low_sub_pf_2020_feb['count_rate'] 

x_jan = low_sub_pf_2020_jan['flash_dur']
y_jan = low_sub_pf_2020_jan['count_rate'] 

x_may = low_sub_pf_2020_may['flash_dur']
y_may = low_sub_pf_2020_may['count_rate']

x_jun = low_sub_pf_2020_jun['flash_dur']
y_jun = low_sub_pf_2020_jun['count_rate'] 

x_jul = low_sub_pf_2020_jul['flash_dur']
y_jul = low_sub_pf_2020_jul['count_rate'] 


low_sub_pf_2012 = low_sub_pf_isr_13078.loc[(low_sub_pf_isr_13078['obsdate'] == '2012-11-15')]
x_2012 = low_sub_pf_2012['flash_dur']
y_2012 = low_sub_pf_2012['count_rate']

p = figure(title = 'Count Rate vs. Flash Duration')
p.y_range = Range1d(2.3, 2.7)
p.circle(x_jul, y_jul, size = 10, color = 'cyan', legend = "July 2020")
p.circle(x_jun, y_jun, size = 10, color = 'magenta', legend = "June 2020")
p.circle(x_may, y_may, size = 10, color = 'pink', legend = "May 2020")
p.circle(x, y, size = 10, color = 'purple', legend = "April 2020") 
p.circle(x_mar, y_mar, size = 10, color = 'blue', legend = "March 2020")
p.circle(x_feb, y_feb, size = 10, color = 'green', legend = "Feb 2020")
p.circle(x_jan, y_jan, size = 10, color = 'orange', legend = "Jan 2020")
p.triangle(x_2012, y_2012, size = 10, color = 'red', legend = "Nov 15th 2012")
p.xaxis.axis_label = "Flash Duration"
p.yaxis.axis_label = "Count Rate"
show(p)

In [33]:
low_sub_pf_2020_aug = low_sub_pf_2020.loc[(low_sub_pf_2020['obsdate'] == '2020-08-03')]
low_sub_pf_2020_sep = low_sub_pf_2020.loc[(low_sub_pf_2020['obsdate'] == '2020-09-08')]
low_sub_pf_2020_oct = low_sub_pf_2020.loc[(low_sub_pf_2020['obsdate'] == '2020-10-06')]

x_aug = low_sub_pf_2020_aug['flash_dur']
y_aug = low_sub_pf_2020_aug['count_rate']

x_sep = low_sub_pf_2020_sep['flash_dur']
y_sep = low_sub_pf_2020_sep['count_rate'] 

x_oct = low_sub_pf_2020_oct['flash_dur']
y_oct = low_sub_pf_2020_oct['count_rate']


low_sub_pf_2012 = low_sub_pf_isr_13078.loc[(low_sub_pf_isr_13078['obsdate'] == '2012-11-15')]
x_2012 = low_sub_pf_2012['flash_dur']
y_2012 = low_sub_pf_2012['count_rate']

p = figure(title = 'Count Rate vs. Flash Duration')
p.y_range = Range1d(2.3, 2.7)
p.circle(x_oct, y_oct, size = 10, color = 'blue', legend = "Oct 2020") 
p.circle(x_sep, y_sep, size = 10, color = 'green', legend = "Sep 2020")
p.circle(x_aug, y_aug, size = 10, color = 'purple', legend = "Aug 2020") 
p.triangle(x_2012, y_2012, size = 10, color = 'red', legend = "Nov 15th 2012")
p.xaxis.axis_label = "Flash Duration"
p.yaxis.axis_label = "Count Rate"
show(p)

Now for Medium Current:

In [34]:
med_sub_pf_isr_13078_nov14 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'MED') & (subarray_mean['propid'] == 13078) & (subarray_mean['shutter'] == 'A') & (subarray_mean['obsdate'] == '2012-11-14')]
med_sub_pf_isr_13078_nov15 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'MED') & (subarray_mean['propid'] == 13078) & (subarray_mean['shutter'] == 'A') & (subarray_mean['obsdate'] == '2012-11-15')]

both_dates = [med_sub_pf_isr_13078_nov14, med_sub_pf_isr_13078_nov15]

med_sub_pf_isr_13078 = pd.concat(both_dates)

In [35]:
med_sub_pf_isr_13078

paths = med_sub_pf_isr_13078.path.tolist() 

means = []
sig_means = []
for f in paths:
    path = f
    data = fits.getdata(path)
    data_clip_1 = stats.sigma_clip(data, 3)
    data_clip_2 = stats.sigma_clip(data_clip_1, 3)
    data_clip_3 = stats.sigma_clip(data_clip_2, 3)
    mean = np.mean(data)
    sig_mean = np.mean(data_clip_3)
    means.append(mean)
    sig_means.append(sig_mean)

dict = {'path': paths, 'mean': means, 'sig_mean': sig_means}
path_and_mean_sub = pd.DataFrame(dict)

FileNotFoundError: [Errno 2] No such file or directory: '/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/ic6da3uqq/ic6da3uqq_flc.fits'

In [36]:
med_sub_pf_isr_13078['count_rate'] = (((med_sub_pf_isr_13078['sig_mean'])*1.55)/med_sub_pf_isr_13078['flash_dur'])

In [37]:
med_sub_pf_isr_13078

,filename,propid,path,obsdate,obstime,shutter,flash_lvl,flash_dur,flash_cur,subarray,exp_time,mean,sig_mean,count_rate
rootname,,,,,,,,,,,,,,
ic6da3uqq,ic6da3uqq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,04:55:38,A,15,0.2,MED,True,0.5,9.771748,9.268235,71.828825
ic6da6wwq,ic6da6wwq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,18:20:36,A,22,0.3,MED,True,0.5,13.886331,13.603044,70.282395
ic6da3usq,ic6da3usq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,04:58:45,A,22,0.3,MED,True,0.5,13.717435,13.400524,69.236038
ic6da6wuq,ic6da6wuq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,18:17:29,A,15,0.2,MED,True,0.5,9.637007,9.300152,72.076181


In [38]:
med_sub_pf_isr_13078.to_csv('countrate_for_2012_isr_med_data.csv')

In [39]:
med_sub_pf_2020_mar = subarray_mean.loc[(subarray_mean['flash_cur'] == 'MED') & (subarray_mean['shutter'] == 'A') & (subarray_mean['propid'] == 15718) & (subarray_mean['obsdate'] == '2020-03-22')]
med_sub_pf_2020_apr = subarray_mean.loc[(subarray_mean['flash_cur'] == 'MED') & (subarray_mean['shutter'] == 'A') & (subarray_mean['propid'] == 15718) & (subarray_mean['obsdate'] == '2020-04-21')]
frames = [med_sub_pf_2020_mar, med_sub_pf_2020_apr]
med_sub_pf_2020 = pd.concat(frames)

In [40]:
med_sub_pf_2020

,filename,propid,path,obsdate,obstime,shutter,flash_lvl,flash_dur,flash_cur,subarray,exp_time,mean,sig_mean
rootname,,,,,,,,,,,,,
ie1b21eqq,ie1b21eqq_flc.fits,15718,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2020-03-22,05:05:18,A,22,0.3,MED,True,0.5,13.770664,13.397955
ie1b21eoq,ie1b21eoq_flc.fits,15718,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2020-03-22,05:02:11,A,15,0.2,MED,True,0.5,9.322201,9.072637
ie1b24g0q,ie1b24g0q_flc.fits,15718,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2020-04-21,01:00:19,A,22,0.3,MED,True,0.5,13.791778,13.401114
ie1b24fyq,ie1b24fyq_flc.fits,15718,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2020-04-21,00:57:12,A,15,0.2,MED,True,0.5,9.527061,9.099190


In [41]:
med_sub_pf_2020['count_rate'] = (((med_sub_pf_2020['sig_mean'])*1.55)/med_sub_pf_2020['flash_dur'])

In [42]:
med_sub_pf_2020
med_sub_pf_2020.to_csv('countrate_for_2020_med_data.csv')

In [ ]:
med_sub_pf_2020_apr = med_sub_pf_2020.loc[(med_sub_pf_2020['obsdate'] == '2020-04-21')]
med_sub_pf_2020_mar = med_sub_pf_2020.loc[(med_sub_pf_2020['obsdate'] == '2020-03-22')]

x = med_sub_pf_2020_apr['flash_dur']
y = med_sub_pf_2020_apr['count_rate']

x_mar = med_sub_pf_2020_mar['flash_dur']
y_mar = med_sub_pf_2020_mar['count_rate']  
    

med_sub_pf_2012 = med_sub_pf_isr_13078.loc[(med_sub_pf_isr_13078['obsdate'] == '2012-11-15')]
x_2012 = med_sub_pf_2012['flash_dur']
y_2012 = med_sub_pf_2012['count_rate']

p = figure(title = 'Count Rate vs. Flash Duration - Medium Current')
#p.y_range = Range1d(2.3, 2.7)
p.circle(x, y, size = 10, color = 'purple', legend_label = "April 21st 2020") 
p.circle(x_mar, y_mar, size = 10, color = 'blue', legend_label = "March 22nd 2020")
p.triangle(x_2012, y_2012, size = 10, color = 'red', legend_label = "2012 Data (Just 11/15)")
p.xaxis.axis_label = "Flash Duration"
p.yaxis.axis_label = "Count Rate"
show(p)